In [1]:
# Hack to import from a parent directory
import sys
path = '..'
if path not in sys.path:
    sys.path.append(path)

In [2]:
#PyData imports
import numpy as np
import pandas as pd
import matplotlib as plt

import stan
import arviz
import os
import nest_asyncio
nest_asyncio.apply()

In [3]:
#Data to be passed to Stan.
state_dim = 4
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100.
prior_scale_factor = 0.25
obs_error_scale = 0.1
obs_every = 10 #Observations every 10 hours.
t = 2000 #Total time span of ODE simulation.
dt = 0.1 #Euler time step size corresponding to data generation. 
x_hat0 = [60.35767864, 5.16483124, 2.0068896, 0.99331202]
y_full = pd.read_csv(os.path.join('generated_data/', 'SAWB-ECA-SS_no_CO2_trunc_short_2021_12_20_18_20_sample_det_y_t_2000_dt_0-01_sd_scale_0-25.csv'))
y = y_full[y_full['hour'] <= t]
y = np.array(y.drop(columns = 'hour')).tolist() #Convert data observations to list of lists to correspond to Stan's array of vectors type.
ts = np.arange(0, t + dt, obs_every).tolist() #Convert array of observation times to list of hours. 
N_t = len(ts)

#Parameter prior means
u_Q_ref_mean = 0.25
Q_mean = 0.001
a_MSA_mean = 0.5
K_DE_mean = 1000
K_UE_mean = 0.1
V_DE_ref_mean = 0.11
V_UE_ref_mean = 0.0044
Ea_V_DE_mean = 40
Ea_V_UE_mean = 30
r_M_mean = 0.0001667
r_E_mean = 0.0002
r_L_mean = 0.0004

In [4]:
data_dict = {
    'state_dim': state_dim,
    'N_t': N_t,
    'ts': ts,
    'y': y,
    'temp_ref': temp_ref,
    'temp_rise': temp_rise,
    'prior_scale_factor': prior_scale_factor,
    'obs_error_scale': obs_error_scale,
    'x_hat0': x_hat0,
    'u_Q_ref_mean': u_Q_ref_mean,
    'Q_mean': Q_mean,
    'a_MSA_mean': a_MSA_mean,
    'K_DE_mean': K_DE_mean,
    'K_UE_mean': K_UE_mean,
    'V_DE_ref_mean': V_DE_ref_mean,
    'V_UE_ref_mean': V_UE_ref_mean,
    'Ea_V_DE_mean': Ea_V_DE_mean,
    'Ea_V_UE_mean': Ea_V_UE_mean,
    'r_M_mean': r_M_mean,
    'r_E_mean': r_E_mean,
    'r_L_mean': r_L_mean
}

In [5]:
AWB_ECA_stan_file = open('AWB-ECA_SS_no_CO2_cont_time.stan').read()
print(AWB_ECA_stan_file)

functions {

  // Temperature function for ODE forcing.
  real temp_func(real t, real temp_ref, real temp_rise) {
    return temp_ref + (temp_rise * t) / (80 * 24 * 365) + 10 * sin((2 * pi() / 24) * t) + 10 * sin((2 * pi() / (24 * 365)) * t);
  }

  // Exogenous SOC input function.
  real i_s_func(real t) {
    return 0.001 + 0.0005 * sin((2 * pi() / (24 * 365)) * t);
  }

  // Exogenous DOC input function.
  real i_d_func(real t) {
    return 0.0001 + 0.00005 * sin((2 * pi() / (24 * 365)) * t);
  }

  // Function for enforcing Arrhenius temperature dependency of ODE parameter.
  real arrhenius_temp(real input, real temp, real Ea, real temp_ref) {
    return input * exp(-Ea / 0.008314 * (1 / temp - 1 / temp_ref));
  }

  // Function for enforcing linear temperature dependency of ODE parameter.
  real linear_temp(real input, real temp, real Q, real temp_ref) {
    return input - Q * (temp - temp_ref);
  }

  /*
  AWB-ECA (equilibrium chemistry approximation) variant of AWB model.
  C[1]

In [6]:
AWB_ECA_stan_model = stan.build(program_code = AWB_ECA_stan_file, data = data_dict, random_seed = 1234)

Building...


Building: Semantic error:   -------------------------------------------------
   158:    // Likelihood evaluation.
   159:    // y ~ normal(x_hat, obs_error_scale * x_hat);
   160:    y ~ normal(x_hat, 1);
           ^
   161:  }
   162:  
   -------------------------------------------------

Ill-typed arguments to '~' statement. No distribution 'normal' was found with the correct signature.

ValueError: Semantic error